Participant Enrollment Overview (v2.1)

First automated version with 2018 updates.
Scott Sutherland (ssutherland@vibrenthealth.com)


In [ ]:
#######################################
print('Setting everything up...')
#######################################

import sys
# Add the libraries folder path to the sys.path list
sys.path.append('../lib/')

import mysql.connector
from mysql.connector import errorcode
from datetime import datetime
from datetime import date
from datetime import time
from datetime import timedelta
import pandas as pd
import numpy as np

import config
from progress import Progress
from ubr import UBRRaceEthnicity, UBRAge, UBRSex, UBRSexualAndGenderMinorities, UBRIncome
from ubr import UBREducation, LoadUBRZipCodes, UBRGeography, UBRAccessToCare, UBRDisability


d = '2019-01-30'
d_prev = '2019-01-29'
w_prev = '2019-01-23'

print('done.')


In [ ]:
######################################
print('Getting the data from the database...')
######################################
connect_options = {
    'user': config.CLOUDSQL_PROD_USER,
    'password': config.CLOUDSQL_PROD_PASSWORD,
    'host': '127.0.0.1',
    'port': '3307',
    'database': 'rdr',
    'raise_on_warnings': True,
}

try:
      cnx = mysql.connector.connect(**connect_options)

except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
else:

    #query = ("DESCRIBE rdr.participant_view;")
    query = ('''   

    SELECT 
    participant_id, 
    hpo, 
    s.google_group pm_site, 
    enrollment_status, 
    withdrawal_status, 
    sex, 
    gender, 
    sexual_orientation, 
    (CASE /*######## ToDo: make sure age_years is based on age at registration ########*/ 
        WHEN age_years IS NULL THEN ''  
        WHEN age_years < 18 THEN '0-17'  
        WHEN age_years < 26 THEN '18-25'  
        WHEN age_years < 36 THEN '26-35'  
        WHEN age_years < 46 THEN '36-45'  
        WHEN age_years < 56 THEN '46-55'  
        WHEN age_years < 66 THEN '56-65'  
        WHEN age_years < 76 THEN '66-75'  
        WHEN age_years < 86 THEN '76-85'  
        ELSE '86+'  END) 
    age_bucket, 
    age_years, /*######## ToDo: get rid of this eventually in favor of age_bucket ######## */
    race, 
    race_codes, 
    hispanic, 
    income, 
    education, 
    pv.state 
    state, 
    pv.zip_code 
    zip_code, 
    LEFT(pv.zip_code, 3) AS zip_code_3_digit, 
    DATE(sign_up_time) signup_date,  
    consent_for_study_enrollment, 
    DATE(consent_for_study_enrollment_time) consent_date,  
    consent_for_electronic_health_records, 
    DATE(consent_for_electronic_health_records_time) ehr_date,  
    questionnaire_on_the_basics, 
    DATE(questionnaire_on_the_basics_time) ppi_thebasics_date,  
    questionnaire_on_overall_health, 
    DATE(questionnaire_on_overall_health_time) ppi_overallhealth_date,  
    questionnaire_on_lifestyle, 
    DATE(questionnaire_on_lifestyle_time) ppi_lifestyle_date,  
    questionnaire_on_healthcare_access, 
    DATE(questionnaire_on_healthcare_access_time) ppi_hc_access_date,  
    questionnaire_on_medical_history, 
    DATE(questionnaire_on_medical_history_time) ppi_med_history_date,  
    questionnaire_on_family_health, 
    DATE(questionnaire_on_family_health_time) ppi_family_health_date,  
    questionnaire_on_medications, 
    DATE(questionnaire_on_medications_time) ppi_medications_date,  
    physical_measurements_status, 
    DATE(physical_measurements_finalized_time) pm_date,  
    DATE(sample_status_1ed04_time) sample_date, 
    DATE(sample_status_1sal_time) 1sal_date,  
    sample_order_status_1sst8, 
    sample_order_status_1pst8, 
    sample_order_status_1hep4, 
    sample_order_status_1ed04, 
    sample_order_status_1ed10, 
    sample_order_status_2ed10, 
    sample_order_status_1cfd9, 
    sample_order_status_1pxr2, 
    sample_order_status_1sal, 
    sample_order_status_1sal2, 
    sample_order_status_1ur10,  
    sample_status_1sst8, 
    sample_status_1pst8, 
    sample_status_1hep4, 
    sample_status_1ed04, 
    sample_status_1ed10, 
    sample_status_2ed10, 
    sample_status_1cfd9, 
    sample_status_1pxr2, 
    sample_status_1sal, 
    sample_status_1sal2, 
    sample_status_1ur10, 
    samples_to_isolate_dna,  

    DATE(sign_up_time) as registered_date,
    DATE(
        CASE WHEN enrollment_status >=2 THEN
            GREATEST( sign_up_time
                , COALESCE(consent_for_study_enrollment_time, '1000-01-01')
                , COALESCE(consent_for_electronic_health_records_time, '1000-01-01')
                )
            ELSE NULL END
        ) member_date,

    DATE(
        CASE WHEN enrollment_status = 3 THEN /*######## ToDo: Make this not rely on ES ########*/
            GREATEST( sign_up_time
                , COALESCE(consent_for_study_enrollment_time, '1000-01-01')
                , COALESCE(consent_for_electronic_health_records_time, '1000-01-01')
                , COALESCE(questionnaire_on_the_basics_time, '1000-01-01')
                , COALESCE(questionnaire_on_overall_health_time, '1000-01-01')
                , COALESCE(questionnaire_on_lifestyle_time, '1000-01-01')
                , COALESCE(physical_measurements_finalized_time, '1000-01-01')
                , CASE WHEN
                         LEAST(
                            COALESCE(sample_status_1ed04_time, '3000-01-01')
                            , COALESCE(sample_status_1sal_time, '3000-01-01')
                            , COALESCE(sample_status_1sal2_time, '3000-01-01')
                            , COALESCE(sample_status_1ed10_time, '3000-01-01')
                            , COALESCE(sample_status_2ed10_time, '3000-01-01')
                            ) = '3000-01-01' THEN NULL
                    ELSE LEAST(
                            COALESCE(sample_status_1ed04_time, '3000-01-01')
                            , COALESCE(sample_status_1sal_time, '3000-01-01')
                            , COALESCE(sample_status_1sal2_time, '3000-01-01')
                            , COALESCE(sample_status_1ed10_time, '3000-01-01')
                            , COALESCE(sample_status_2ed10_time, '3000-01-01')
                            )    
                    END
                )
            ELSE NULL END
        ) fp_date


    FROM rdr.participant_view pv 
    LEFT OUTER JOIN rdr.site s on s.site_id = pv.physical_measurements_finalized_site_id 
    WHERE 
        withdrawal_status = 1 
    ORDER BY sign_up_time ;

    ''')
    

    #Now execute the query and put it it into a dataframe
    myData = pd.read_sql(query, 
                         con=cnx,  
                         parse_dates=['signup_date', 'consent_date', 'ehr_date', 'ppi_thebasics_date', 
                                      'ppi_overallhealth_date', 'ppi_lifestyle_date', 'ppi_med_history_date', 
                                      'ppi_hc_access_date', 'ppi_family_health_date', 'ppi_medications_date', 
                                      'pm_date', 'sample_date', '1sal_date', 
                                      'registered_date', 'member_date', 'fp_date'])
    
    #Close the connection
    cnx.close()
    
    print(myData.shape[0], 'records received.')

In [ ]:
#######################################
print('Adding extra columns for searching, sorting, and filtering...')
#######################################

## Add the needed columns to our dataframe

#Add columns for the UBR fields and columns needed for the PEO
myData.insert(loc=0, column = 'UBR', value = 0)
myData.insert(loc=0, column = 'UBRScore', value = 0)
myData.insert(loc=0, column = 'UBRCategory', value = '')
myData.insert(loc=0, column = 'UBRMultipleCategories', value = '')
myData.insert(loc=0, column = 'UBR1_RaceEthnicity', value = 0)
myData.insert(loc=0, column = 'UBR2_Age', value = 0)
myData.insert(loc=0, column = 'UBR3_Sex', value = 0)
myData.insert(loc=0, column = 'UBR4_SexualAndGenderMinorities', value = 0)
myData.insert(loc=0, column = 'UBR5_Income', value = 0)
myData.insert(loc=0, column = 'UBR6_Education', value = 0)
myData.insert(loc=0, column = 'UBR7_Geography', value = 0)
myData.insert(loc=0, column = 'UBR8_AccessToCare', value = 0)
myData.insert(loc=0, column = 'UBR9_Disability', value = 0)
myData.insert(loc=0, column = 'SimplifiedRace', value = '')
myData.insert(loc=0, column = 'Ancestry', value = '')
myData.insert(loc=0, column = 'in_includelist', value = 0)
myData.insert(loc=0, column = 'in_excludelist', value = 0)
myData.insert(loc=0, column = 'in_exclude_ghosts_list', value = 0)


print('done.')

In [ ]:
#######################################
print("Joining in additional columns we didn't get in the primary SQL...")
#######################################

# Do this: 
#     load dataframe that includes timestamp called "response_time"
#     Sort it: dataToJoin.sort_values("response_time", inplace = True)
#     Drop dups: dataToJoin.drop_duplicates(subset ="participant_id", keep = "first", inplace = True)
#     Then do the join, only keeeping the colums we want (not timestamp)

print('myData dimensions: {}'.format(myData.shape))


#########
print('• Loading DV EHR Intent data')
#########
connect_options = {
    'user': config.CLOUDSQL_PROD_USER,
    'password': config.CLOUDSQL_PROD_PASSWORD,
    'host': '127.0.0.1',
    'port': '3307',
    'database': 'rdr',
    'raise_on_warnings': True,
}

try:
      cnx = mysql.connector.connect(**connect_options)
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
else:
    query = ('''   
    SELECT
        p.participant_id,
        ac.value AS dv_ehr_intent_status,
        qr.created AS dv_ehr_intent_time
    FROM
        rdr.participant p
        INNER JOIN rdr.questionnaire_response qr ON p.participant_id = qr.participant_id
        INNER JOIN rdr.questionnaire_response_answer qra
           ON qra.questionnaire_response_id = qr.questionnaire_response_id
        INNER JOIN rdr.questionnaire_question qq ON qra.question_id = qq.questionnaire_question_id
        INNER JOIN rdr.questionnaire q ON qq.questionnaire_id = q.questionnaire_id
        INNER JOIN rdr.code qc ON qq.code_id = qc.code_id
        INNER JOIN rdr.participant_summary ps ON p.participant_id = ps.participant_id
        LEFT OUTER JOIN rdr.hpo ON p.hpo_id = hpo.hpo_id
        LEFT OUTER JOIN rdr.code ac ON qra.value_code_id = ac.code_id
    WHERE 
        p.withdrawal_status < 2 AND /* NOT_WITHDRAWN*/
        qc.value = 'DVEHRSharing_AreYouInterested' AND 
        (ps.email IS NULL OR ps.email NOT LIKE '%@example.com') AND
        (hpo.name IS NULL OR hpo.name != 'TEST'); 
    ''')
    
    #Now execute the query and put it it into a dataframe
    dataToJoin = pd.read_sql(query, con=cnx, parse_dates=['dv_ehr_intent_time'])
    
    #Close the connection
    cnx.close()

print('data-file dimensions: {}'.format(dataToJoin.shape))

## Old version, reading from data file
# inFilePath = '2018-12-28_DV_EHR_intent_data.csv'    
# dataToJoin = pd.read_csv(inFilePath, parse_dates=['dv_ehr_intent_time'])
dataToJoin.sort_values("dv_ehr_intent_time", inplace = True)    #Sort it first
dataToJoin.drop_duplicates(subset ="participant_id", keep = "last", inplace = True)   #Keep the final value for a given PID if there are duplicates 
myData = pd.merge(myData, dataToJoin, how='left', on='participant_id', validate='one_to_one') #merge, checking to make sure join is 1:1 (that is, no dups in file being merged)
print('result dimensions: {}'.format(myData.shape))

# print()

#########
print('• Loading DV Market data')
#########

inFilePath = '../data/2018-12-13-markets.csv'    
dataToJoin = pd.read_csv(inFilePath, dtype={'zip_3_digit': str})
print('data-file dimensions: {}'.format(dataToJoin.shape))
#dataToJoin.sort_values("dv_ehr_intent_time", inplace = True)    #Sort it first
#dataToJoin.drop_duplicates(subset ="participant_id", keep = "last", inplace = True)   #Keep the final value for a given PID if there are duplicates 
myData = pd.merge(myData, dataToJoin, how='left', left_on='zip_code_3_digit', right_on='zip_3_digit', validate='many_to_one') #merge, checking to make sure join is 1:1 (that is, no dups in file being merged)
print('result dimensions: {}'.format(myData.shape))

In [ ]:
#######################################
print('Filling the columns with calculated values...')
#######################################

#Set this value to setermine whether women are counted in the UBR categories or not
womenAreUBR = False

#Set the file path to the UBR zip codes, and uncommnt the line below
zipcodesFilePath = '../data/Rural classified zip codes US 1 5 18.csv'
zips = pd.read_csv(zipcodesFilePath)

#get the total number of participants for use later
participantCount = myData.shape[0]

#set up progress bar
p = Progress(participantCount,10)

#Iterate through the participants and calculate fields
sexCount = 0
for index, row in myData.iterrows():
    #ToDo: Change this next line when we have access to age at consent.
    myData.at[index,'UBR1_RaceEthnicity'] = UBRRaceEthnicity(row['race_codes'], row['hispanic']) #set UBR1_RaceEthnicity field to value returned from UBRRaceEthnicity() 
    myData.at[index,'UBR2_Age'] = UBRAge(row['age_years']) #set UBR2_Age field to value returned from UBRAge() 
    myData.at[index,'UBR3_Sex'] = UBRSex(row['sex'], womenAreUBR) #set UBR3_Sex field to value returned from UBRSex() 
    myData.at[index,'UBR4_SexualAndGenderMinorities'] = UBRSexualAndGenderMinorities(row['sexual_orientation'], row['gender'], row['sex'], womenAreUBR) #set UBR4_SexualAndGenderMinorities field to value returned from UBRSexualAndGenderMinorities() 
    myData.at[index,'UBR5_Income'] = UBRIncome(row['income']) #set UBR5_Income field to value returned from UBRIncome() 
    myData.at[index,'UBR6_Education'] = UBREducation(row['education']) #set UBR6_Education field to value returned from UBREducation() 
    myData.at[index,'UBR7_Geography'] = UBRGeography(row['zip_code'], zips.ZIP.values) #set UBR7_Geography field to value returned from UBRGeography() 
    #myData.at[index,'UBR8_AccessToCare'] = UBRAccessToCare() #set UBR7_Geography field to value returned from UBRGeography() 
    #myData.at[index,'UBR9_Disability'] = UBRDisability() #set UBR7_Geography field to value returned from UBRGeography() 

    
    
    #set UBRScore to the sum of the UBR individual scores
    myData.at[index,'UBRScore'] = (myData['UBR1_RaceEthnicity'][index] + 
                                          myData['UBR2_Age'][index] + 
                                          myData['UBR3_Sex'][index] +
                                          myData['UBR4_SexualAndGenderMinorities'][index] + 
                                          myData['UBR5_Income'][index] + 
                                          myData['UBR6_Education'][index] + 
                                          myData['UBR7_Geography'][index] + 
                                          myData['UBR8_AccessToCare'][index] + 
                                          myData['UBR9_Disability'][index]
                                         ) 
    
    #Set UBR to 1 if _one_or_more_ UBR categories are set. Otherwise set it to 0
    if myData['UBRScore'][index] > 0:
        myData.at[index,'UBR'] = 1
    else:
        myData.at[index,'UBR'] = 0
        
    #Set UBRCategory to the category set if only qualified in 1 UBR category
    # set to "Multiple Categories" if more than one category is set. 
    # set to "Not UBR" if no categories are set.
    
    ubrCategories = [
        'UBR9_Disability',
        'UBR8_AccessToCare',
        'UBR7_Geography',
        'UBR6_Education',
        'UBR5_Income',
        'UBR4_SexualAndGenderMinorities',
        'UBR3_Sex',
        'UBR2_Age',
        'UBR1_RaceEthnicity']
    
    if myData['UBRScore'][index] == 0:
        myData.at[index,'UBRCategory'] = "Not UBR"
    elif myData['UBRScore'][index] > 1:
        myData.at[index,'UBRCategory'] = "Multiple_Categories"
        for category in ubrCategories:
            if myData[category][index] == 1:
                myData.at[index,'UBRMultipleCategories'] += (category + ",")
    else:
        for category in ubrCategories:
            if myData[category][index] == 1:
                myData.at[index,'UBRCategory'] = category
        
   

    #Increase the status bar
    p.increase()
    
#print(c_1)
print()
print('done.')

In [ ]:
#######################################
print('Applying the inclusion and exclusion lists...')
#######################################

###
# Get the includelist PIDs and excludelist PIDs
###
filePath = 'All_known_pids.csv'
#filePath = '2018-09-17_PIDs_for_Holly.csv'
includelist = pd.read_csv(filePath)

filePath = '../data/2018-08-20-non_participant_PIDs.txt'
excludelist = pd.read_csv(filePath)

filePath = '../data/ghost_participants_2019-01-31.csv'
exclude_ghosts_list = pd.read_csv(filePath)


####
# Fill the in_includelist, in_excludelist columns
####

for index, row in myData.iterrows():
    #set the includelist column value
    if int(row['participant_id']) in includelist.AOU_PID.values:
        myData.at[index,'in_includelist'] = 1
    else:
        myData.at[index,'in_includelist'] = 0
        
    #set the in_excludelist column value
    if int(row['participant_id']) in excludelist.participant_id.values:
        myData.at[index,'in_excludelist'] = 1
    else:
        myData.at[index,'in_excludelist'] = 0
        
    #set the in_exclude_ghosts_list column value
    if int(row['participant_id']) in exclude_ghosts_list.participant_id.values:
        myData.at[index,'in_exclude_ghosts_list'] = 1
    else:
        myData.at[index,'in_exclude_ghosts_list'] = 0
        
####
# See how many we found
####
print(myData.shape[0], 'participant records received.')
print(includelist.shape[0], 'includelist records received.')
print(excludelist.shape[0], 'excludelist records received.')

print('Total ghosts found:', myData[(myData['in_includelist'] == 0)].shape[0])

print('Total participants in includelist:', myData[(myData['in_includelist'] == 1)].shape[0])
print('Total participants NOT in excludelist:', myData[(myData['in_excludelist'] == 0)].shape[0])
print('Total participants IN includelist and NOT in excludelist:', myData[
        (myData['in_includelist'] == 1) & 
        (myData['in_excludelist'] == 0)
    ].shape[0])
#print('Total Ppts with EHR:', myData[(filtered_data['has_EHR'] == 1)].shape[0])

####
#Finally, apply the filters to get the new, filtered dataset
####
how = 'use_ghost_list' # should = 'use_pid_master_list' only if you want to read a whole master list of PIDs

if how == 'use_pid_master_list':
    print('removing any ghosts not in master list')
    filtered_data = myData.loc[
        (myData['in_includelist'] == 1) & 
        (myData['in_excludelist'] == 0) ]
else:  # 'use_ghosts'
    print('removing specific ghosts in list')
    filtered_data = myData.loc[
        (myData['in_exclude_ghosts_list'] == 0) & 
        (myData['in_excludelist'] == 0) ]
    
print('Count of filtered participants:', filtered_data.shape[0])
print('done.')

In [ ]:
hpos = sorted(filtered_data.hpo.unique())
site_names = pd.read_csv('../data/sites.csv')
sorted_hpos_ids = site_names.hpo_id.unique()


######################################
print('Generating PEO Data...')
######################################
print()

## Heading
print(' ', sep='', end='')
for hpo_id in sorted_hpos_ids:
    short_name = site_names[site_names['hpo_id'].str.match(hpo_id)].values.flatten().tolist()[3]
    hpo = site_names[site_names['hpo_id'].str.match(hpo_id)].values.flatten().tolist()[1]
    print(',', short_name, sep='', end='')
print()

    
### All Participants ###
print('All', sep='', end='')
for hpo_id in sorted_hpos_ids:
    short_name = site_names[site_names['hpo_id'].str.match(hpo_id)].values.flatten().tolist()[3]
    hpo = site_names[site_names['hpo_id'].str.match(hpo_id)].values.flatten().tolist()[1]
    
    all_Ppts_hpo = filtered_data[
        (filtered_data['hpo'] == hpo) &
        (filtered_data['registered_date'] <= d) 
        ].shape[0]
    all_Ppts_prev_hpo = filtered_data[
        (filtered_data['hpo'] == hpo) &
        (filtered_data['registered_date'] <= d_prev) 
        ].shape[0]
    
    print(',', all_Ppts_hpo, sep='', end='')
print()

### Since last report ###
print('24h', sep='', end='')
for hpo_id in sorted_hpos_ids:
    short_name = site_names[site_names['hpo_id'].str.match(hpo_id)].values.flatten().tolist()[3]
    hpo = site_names[site_names['hpo_id'].str.match(hpo_id)].values.flatten().tolist()[1]
    
    all_Ppts_hpo = filtered_data[
        (filtered_data['hpo'] == hpo) &
        (filtered_data['registered_date'] <= d) 
        ].shape[0]
    all_Ppts_prev_hpo = filtered_data[
        (filtered_data['hpo'] == hpo) &
        (filtered_data['registered_date'] <= d_prev) 
        ].shape[0]
    
    print(',', all_Ppts_hpo - all_Ppts_prev_hpo, sep='', end='')
print()

### Core Participants ###
print('Core', sep='', end='')
for hpo_id in sorted_hpos_ids:
    short_name = site_names[site_names['hpo_id'].str.match(hpo_id)].values.flatten().tolist()[3]
    hpo = site_names[site_names['hpo_id'].str.match(hpo_id)].values.flatten().tolist()[1]
    
    core_Ppts_hpo = filtered_data[
        (filtered_data['hpo'] == hpo) &
        (np.isnat(filtered_data['fp_date']) == False) &
        (filtered_data['fp_date'] <= d) 
        ].shape[0]
    
    print(',', core_Ppts_hpo, sep='', end='')
print()

### Members ###
print('Members', sep='', end='')
for hpo_id in sorted_hpos_ids:
    short_name = site_names[site_names['hpo_id'].str.match(hpo_id)].values.flatten().tolist()[3]
    hpo = site_names[site_names['hpo_id'].str.match(hpo_id)].values.flatten().tolist()[1]
    
    member_Ppts_hpo = filtered_data[
        (filtered_data['hpo'] == hpo) &
        (np.isnat(filtered_data['member_date']) == False) &
        (filtered_data['member_date'] <= d) &
        ((np.isnat(filtered_data['fp_date']) | (filtered_data['fp_date'] > d))) 
        ].shape[0]
    
    print(',', member_Ppts_hpo, sep='', end='')
print()

### Registered ###
print('Registered', sep='', end='')
for hpo_id in sorted_hpos_ids:
    short_name = site_names[site_names['hpo_id'].str.match(hpo_id)].values.flatten().tolist()[3]
    hpo = site_names[site_names['hpo_id'].str.match(hpo_id)].values.flatten().tolist()[1]
    
    registered_Ppts_hpo = filtered_data[
        (filtered_data['hpo'] == hpo) &
        (filtered_data['registered_date'] <= d) &
        ((np.isnat(filtered_data['member_date'])) | (filtered_data['member_date'] > d))
        ].shape[0]
    
    print(',', registered_Ppts_hpo, sep='', end='')
print()

### Gender ID ###

genderIDs = pd.read_csv('../data/gender.csv')
sorted_genderIDs = genderIDs.gender_id.unique()

#Iterate through gender IDs and then within each, iterate through sites        
for genderID in sorted_genderIDs:
    g_id = genderIDs[genderIDs['gender_id'].str.match(genderID)].values.flatten().tolist()[0]
    label = genderIDs[genderIDs['gender_id'].str.match(genderID)].values.flatten().tolist()[1]
    print(label, end='')
    for hpo_id in sorted_hpos_ids:
        short_name = site_names[site_names['hpo_id'].str.match(hpo_id)].values.flatten().tolist()[3]
        hpo = site_names[site_names['hpo_id'].str.match(hpo_id)].values.flatten().tolist()[1]
        
        #Gender Identity
        if genderID == 'None':
            all_Ppts_gender = filtered_data[
                (filtered_data['hpo'] == hpo) &
                pd.isnull(filtered_data['gender']) &
                (filtered_data['registered_date'] <= d)
                ].shape[0]
            print(',', all_Ppts_gender, sep='', end='')
        else:    
            all_Ppts_gender = filtered_data[
                (filtered_data['hpo'] == hpo) &
                (filtered_data['gender'] == g_id) &
                (filtered_data['registered_date'] <= d)
                ].shape[0]
            print(',', all_Ppts_gender, sep='', end = '')
    print()
    
### Race ###

races = pd.read_csv('../data/race.csv')
sorted_races = races.race.unique()

#Iterate through races and then within each, iterate through sites        
for race in sorted_races:
    race_id = races.loc[races['race'] == race].values.flatten().tolist()[0]
    label = races.loc[races['race'] == race].values.flatten().tolist()[2]
    print(label, end='')
    for hpo_id in sorted_hpos_ids:
        short_name = site_names[site_names['hpo_id'].str.match(hpo_id)].values.flatten().tolist()[3]
        hpo = site_names[site_names['hpo_id'].str.match(hpo_id)].values.flatten().tolist()[1]
    
        if race == 'None':
            all_Ppts_race = filtered_data[
                (filtered_data['hpo'] == hpo) &
                pd.isnull(filtered_data['race']) &
                (filtered_data['registered_date'] <= d)
                ].shape[0]
            print(',', all_Ppts_race, sep='', end='')
        else:    
            all_Ppts_race = filtered_data[
                (filtered_data['hpo'] == hpo) &
                (filtered_data['race'] == race_id) &
                (filtered_data['registered_date'] <= d)
                ].shape[0]
            print(',', all_Ppts_race, sep='', end = '')
    print()

### Age Buckets ###

ages = pd.read_csv('../data/age.csv')
sorted_ages = ages.age.unique()
        
#Iterate through ages and then within each, iterate through sites        
for age in sorted_ages:
    age_id = ages[ages['age'].str.match(age)].values.flatten().tolist()[0]
    label = ages[ages['age'].str.match(age)].values.flatten().tolist()[1]
    print(label, end='')
    for hpo_id in sorted_hpos_ids:
        short_name = site_names[site_names['hpo_id'].str.match(hpo_id)].values.flatten().tolist()[3]
        hpo = site_names[site_names['hpo_id'].str.match(hpo_id)].values.flatten().tolist()[1]
    
        if age == 'None':
            all_Ppts_age = filtered_data[
                (filtered_data['hpo'] == hpo) &
                pd.isnull(filtered_data['age_bucket']) &
                (filtered_data['registered_date'] <= d)
                ].shape[0]
            print(',', all_Ppts_age, sep='', end='')
        else:    
            all_Ppts_age = filtered_data[
                (filtered_data['hpo'] == hpo) &
                (filtered_data['age_bucket'] == age_id) &
                (filtered_data['registered_date'] <= d)
                ].shape[0]
            print(',', all_Ppts_age, sep='', end = '')
    print()
print()


print()

print('done.')
